# Competition: starter notebook

This notebook gives a starting point for the competition: it loads the data, picks a few features, adds a DTC on top and gives a baseline accuracy of 61%. 

There is a **lot** of room for improvement! This is where you come in. Go through the notebook and try to gradually improve the notebook in order to get a better accuracy on the test set. 

**Tasks**:

1. Run through this notebook, make sure you understand the general structure
2. Run through the file `predictive_model.py`, make sure you understand how this notebook translates to the `predictive_model.py` file
3. Modify the notebook to increase the performances as much as you can
4. When you're ready to submit, run the cell to save your trained model (see at the bottom of this notebook) and adapt the `predictive_model.py` file to correspond to what you did in the notebook.

## Submitting your model

When you would like to submit your model, copy paste the relevant steps from the notebook to the auxiliary file `predictive_model.py`. **Test** that the whole procedure worked by typing

```bash
python model_tester.py
```

if it returns something like 

```
Accuracy score (dummy test set): 0.6159
MODEL RAN SUCCESSFULLY
```

it's a good sign. If it failed then you need to fix it before submitting!

When submitting, make sure you send **both** the `predictive_model.py` file as well as the auxiliary `predictive_model.pickle` file that this notebook should generate (see end of notebook) and which contains your trained classifier.


## Important note

If, in your feature engineering, you create categories for specific occurences of words for example. Think about how to deal with a case where a word is in the test data and **not** in the training data.

This is very important as your model will likely fail on the test data otherwise.

## Importing the libraries

Below we import some useful libraries to build an elementary model, feel free to add whatever libraries you need (make sure to include them in `predictive_model.py`). 

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.tree import DecisionTreeClassifier

### Getting the data

The data corresponds to kickstarter projects, data about the projects and whether it was successfully funded or not. 

In [2]:
kick = pd.read_csv("kick.csv", low_memory=False)
kick.head(3)

,photo,name,blurb,goal,state,slug,disable_communication,country,currency,currency_symbol,...,created_at,launched_at,static_usd_rate,creator,location,category,profile,urls,source_url,friends
0,"{""small"":""https://ksr-ugc.imgix.net/assets/012...",Angular - Where Modern Art meets Cards,Angular is a minimalist card design for simpli...,17380.0,failed,angular-where-modern-art-meets-cards,False,US,USD,$,...,1455845363,1456694829,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""US"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN
1,"{""small"":""https://ksr-ugc.imgix.net/assets/014...",Ladybeard is KAWAII-CORE,Original songs and music videos to jump start ...,24000.0,failed,ladybeard-is-kawaii-core,False,US,USD,$,...,1475568868,1480946454,1.000000,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""JP"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN
2,"{""small"":""https://ksr-ugc.imgix.net/assets/011...",Vegan Cafe Delivery Service in Vancouver BC,Our project is to launch a vegan lunch deliver...,40000.0,failed,vegancafeca,False,CA,CAD,$,...,1405218883,1405957628,0.926746,"{""urls"":{""web"":{""user"":""https://www.kickstarte...","{""country"":""CA"",""urls"":{""web"":{""discover"":""htt...","{""urls"":{""web"":{""discover"":""http://www.kicksta...","{""background_image_opacity"":0.8,""should_show_f...","{""web"":{""project"":""https://www.kickstarter.com...",https://www.kickstarter.com/discover/categorie...,NaN


### Having a first look at the data

In [3]:
# All the columns that are available
print("COLUMNS:\n")
print(kick.columns)
# The dimension of the dataset
print("\nDIMENSIONS:\n")
print(kick.shape)
# The types of all the columns
print("\nTYPES FOR THE COLUMNS:\n")
print(kick.dtypes)

COLUMNS:

Index(['photo', 'name', 'blurb', 'goal', 'state', 'slug',
       'disable_communication', 'country', 'currency', 'currency_symbol',
       'currency_trailing_code', 'deadline', 'state_changed_at', 'created_at',
       'launched_at', 'static_usd_rate', 'creator', 'location', 'category',
       'profile', 'urls', 'source_url', 'friends'],
      dtype='object')

DIMENSIONS:

(112879, 23)

TYPES FOR THE COLUMNS:

photo                      object
name                       object
blurb                      object
goal                      float64
state                      object
slug                       object
disable_communication        bool
country                    object
currency                   object
currency_symbol            object
currency_trailing_code       bool
deadline                    int64
state_changed_at            int64
created_at                  int64
launched_at                 int64
static_usd_rate           float64
creator                    object

### Extracting the response

The response is the "state" column, we extract it, and encode it:

* `failed`, `canceled`, `suspended` --> 0
* `successful` --> 1

**Note**: do not modify this. 

In [4]:
# Do not modify this cell
response = kick["state"]
del kick["state"]
response = response.apply(lambda x: 0 if x in ['failed', 'canceled', 'suspended'] else 1)

Now the projects are labelled as either 0 or 1, which you can check by running the next cell:

In [5]:
response.unique()

array([0, 1], dtype=int64)

## A first, simple model

In this model, we extract the numerical columns and apply a Decision Tree Classifier. 

In [6]:
numerical_columns = kick.select_dtypes(include=['float64', 'int64']).columns

Train test split after only considering the columns extracted above

In [7]:
X_train, X_test, y_train, y_test = train_test_split(kick[numerical_columns],
                                                    response, random_state=143)

Fitting a DTC with maximum depth of 6

In [8]:
dtc = DecisionTreeClassifier(max_depth=6)

dtc.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

Getting the prediction on the test set with the DTC and comparing to the truth.

In [9]:
y_pred = dtc.predict(X_test)

print("ACCURACY ON TEST SET: {0:.3f}".format(accuracy_score(y_test, y_pred)))
print("CLASSIFICATION REPORT:\n", 
      str(classification_report(y_test, y_pred, digits=3)))

ACCURACY ON TEST SET: 0.616
CLASSIFICATION REPORT:
              precision    recall  f1-score   support

          0      0.622     0.720     0.667     15110
          1      0.605     0.496     0.545     13110

avg / total      0.614     0.616     0.610     28220



## OUTPUT YOUR MODEL

Adapt and run this cell when you have trained your classifier and whish to submit it. 

Pickle allows to save python objects preserving their structure.

In [10]:
import pickle

Only change the name of the classifier (here `dtc`) to the name of your trained model, leave the rest as is. 

In [12]:
with open('predictive_model.pickle', 'wb') as output_file:
    # Change what is between the square brackets, nothing else
    pickle.dump([dtc], output_file)